In [1]:
! pip install opencv-python

In [2]:
import pandas as pd
import numpy as np
import cv2
import os, sys
from tqdm import tqdm

In [3]:
! pip install tqdm

In [3]:
import os
import requests,zipfile,io

In [3]:
os.getcwd()

'/gpfs/global_fs01/sym_shared/YPProdSpark/user/se4b-ad22e222bc94c1-80e192e0f0b2/notebook/work'

In [4]:
url = requests.get('https://he-s3.s3.amazonaws.com/media/hackathon/deep-learning-challenge-1/identify-the-objects/a0409a00-8-dataset_dp.zip')
data = zipfile.ZipFile(io.BytesIO(url.content))
data.extractall()

KeyboardInterrupt: 

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (64,64))
    return img

In [6]:
TRAIN_PATH = 'train_img/'
TEST_PATH = 'test_img/'

In [7]:
train_img, test_img = [],[]
for img_path in tqdm(train['image_id'].values):
    train_img.append(read_img(TRAIN_PATH + img_path + '.png'))

for img_path in tqdm(test['image_id'].values):
    test_img.append(read_img(TEST_PATH + img_path + '.png'))

100%|██████████| 1732/1732 [00:32<00:00, 53.06it/s]


In [8]:
x_train = np.array(train_img, np.float32) / 255.
x_test = np.array(test_img, np.float32) / 255.

In [9]:
# target variable - encoding numeric value
label_list = train['label'].tolist()
Y_train = {k:v+1 for v,k in enumerate(set(label_list))}
y_train = [Y_train[k] for k in label_list]   
y_train = np.array(y_train)

In [10]:
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [11]:
y_train = to_categorical(y_train)

In [12]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

In [13]:
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(y_train.shape[1], activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [14]:
batch_size = 10 # tune it
epochs = 75 # increase it

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_train)

In [ ]:
history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    callbacks=[ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]
)

Epoch 1/75
321/321 [==============================] - 354s - loss: 3.0221 - acc: 0.1436   

/usr/local/src/conda3_runtime.v17/4.1.1/lib/python3.5/site-packages/keras/callbacks.py:405: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)



Epoch 2/75
321/321 [==============================] - 346s - loss: 2.4527 - acc: 0.2869   
Epoch 3/75
321/321 [==============================] - 346s - loss: 2.0531 - acc: 0.3916   
Epoch 4/75
321/321 [==============================] - 349s - loss: 1.7636 - acc: 0.4822   
Epoch 5/75
321/321 [==============================] - 349s - loss: 1.5603 - acc: 0.5274   
Epoch 6/75
321/321 [==============================] - 350s - loss: 1.3835 - acc: 0.5857   
Epoch 7/75
321/321 [==============================] - 349s - loss: 1.2489 - acc: 0.6137   
Epoch 8/75
321/321 [==============================] - 357s - loss: 1.0973 - acc: 0.6623   
Epoch 9/75
321/321 [==============================] - 364s - loss: 0.9493 - acc: 0.7090   
Epoch 10/75
321/321 [==============================] - 375s - loss: 0.9016 - acc: 0.7174   
Epoch 11/75
321/321 [==============================] - 392s - loss: 0.7497 - acc: 0.7620   
Epoch 12/75
321/321 [==============================] - 372s - loss: 0.5992 - acc: 0.809

In [ ]:
predictions = model.predict(x_test)

In [ ]:
predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in Y_train.items()}
pred_labels = [rev_y[k] for k in predictions]

In [ ]:
sub = pd.DataFrame({'image_id':test.image_id, 'label':pred_labels})
sub.to_csv('sub_vgg.csv', index=False) ## ~0.59